# ML Week 3 - Cross Validation

---

[Top](#ML-Week-3---Cross-Validation) | [Previous section](#ML-Week-3---Cross-Validation) | [Next section](#Part-0:-Quick-review) | [Bottom](#Thank-you)

This notebook has the following sections:

* [Part 0: Quick review!](#Part-0:-Quick-review)
* [Part 1: Introduction to cross-validation](#Part-1:-Introduction-to-cross-validation)
* [Part 2: Figuring out the correct complexity - regularisation](#Part-2:-Figuring-out-the-correct-complexity---regularisation)
* [Part 3: K-Fold Cross Validation](#Part-3:-K-Fold-Cross-Validation)
* [Part 4: Try it yourself](#Part-4:-Try-it-yourself)


## Part 0: Quick review
---

[Top](#ML-Week-3---Cross-Validation) | [Previous section](#ML-Week-3---Cross-Validation) | [Next section](#Part-1:-Introduction-to-cross-validation) | [Bottom](#Thank-you)

Let's load up the full [capital bikeshare](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset) dataset from last week. We can also use the [`pd.DataFrame.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) method to begin to understand our data. 

There is a slight difference within this dataset compared to last week. The `temp` column describes a `normalised` temperature, meaning all temperature values have been **scaled to be between 0 and 1**.

In [ ]:
# Import pandas
import pandas as pd

# Load data
bikeshare_data = pd.read_csv('data/day.csv')

# Describe data
bikeshare_data.describe().transpose()

Let's import some other useful libraries for linear regression.

In [ ]:
# Import the sklearn linear regression function
from sklearn.linear_model import LinearRegression

# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import metrics
from sklearn.metrics import mean_squared_error, r2_score

# Import train_test_split
from sklearn.model_selection import train_test_split

### Exercise

The following code cell uses the [sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) object to create a function that runs linear regression to predict the `cnt` of people using bikes on a specific day. It then outputs the model.

Remember last week how we can raise variables to different powers and increase the complexity of our model. The function defined below allows us to raise any variable by any power. It does this because its input are two different python lists.

* The `cols` input is a list of columns that will be used as features for the model. For example, if `cols` is `['workingday', 'hum']`, the regression will use these two columns as features
* The `powers` input is a list of the maximum power for each of the columns to be raised to within the model

For example, the following function call will create the following linear equation...

```python
# Inputs
cols = ['workingday', 'hum']
powers = [3, 2]

model = run_linear_regression(data, cols=cols, powers=powers, target='cnt')
```

Defined as follows...

$cnt = m_{workingday}x_{workingday} + m_{workingday^{2}}x^{2}_{workingday} + m_{workingday^{3}}x^{3}_{workingday} + m_{hum}x_{hum} + m_{hum^{2}}x^{2}_{hum} + b$

The model will print the resulting MSE, and correlation of the output on a **test set**.

There are **gaps** in the function, indicated by comments, specifically where it says `INSERT CODE: ...`. Fill in these gaps.

In [ ]:
def run_linear_regression(data, cols, powers, target='cnt'):
    """
    Run linear regression on a given feature set, raising to a given power, and predicting
    a given target
    
    :param data: <pd.DataFrame>, the data to be used for linear regression
    :param cols: <list>, the columns to use as features
    :param powers: <list>, the parameters to use as powers
    :param target: <str>, the target variable, defaults to 'cnt' assuming the
                   correct preprocessing
    
    :returns: the trained LinearRegression model
    """
    # Create necessary columns
    model_data = data.copy()
    
    # Go through each column and add the correct powers
    for i in range(len(cols)):
        # Check if power is > 1
        if powers[i] > 1:
            for p in range(2, powers[i] + 1):
                model_data[cols[i] + '_' + str(p)] = model_data[cols[i]] ** p
                # APPEND
                cols.append(cols[i] + '_' + str(p))
        
    
    # Create a train test split
    train, test = train_test_split(model_data, random_state=42)
    
    # 1. INSERT CODE: CREATE A LINEAR REGRESSION OBJECT AND STORE THE OBJECT IN A VARIABLE CALLED "lr"
    lr = 0
    
    # 2. INSERT CODE: FIT THE LINEAR REGRESSION MODEL TO THE TRAIN DATA, FITTING COLUMNS IN "cols"
    
    
    # 3. INSERT CODE: PREDICT THE TEST DATA, USING THE COLUMNS in "cols", AND SAVE TO A VARIABLE
    #    CALLED Y_PRED
    y_pred = 0

    
    # 4. INSERT CODE: CALCULATE THE MSE USING THE TEST DATA, AND SAVE TO A VARIABLE CALLED "mse"
    #    REMEMBER TO PREDICT USING ONLY THE COLUMNS DEFINED IN "cols"
    mse = 0
    
    
    # Calculate the correlation coefficient
    r2 = r2_score(test[target], y_pred)
    
    # Print results
    print('The MSE is: %.2f' % mse)
    print('The r2 is: %.2f' % r2)
    
    # return model
    return lr

The following cells will run the result.

In [ ]:
# Setup columns
cols = ['workingday', 'hum']
powers = [3, 2]

# Run the model
run_linear_regression(data=bikeshare_data, cols=cols, powers=powers, target='cnt')

## Part 1: Introduction to cross-validation

---
[Top](#ML-Week-2---Linear-Regression) | [Previous section](#Part-0:-Quick-review) | [Next section](#Part-2:-Figuring-out-the-correct-complexity---regularisation) | [Bottom](#Thank-you)

### Review from last week

Last week we began to talk about a new way to test the "fit" of a model to data. Remember that we assessed model fit using the **mean squared error**, which measured the average squared distance between our dataset and the model fit to this data.

The new method tested the model fit not on the training data, but on a **set of data we held out** of the data, called the **test dataset**.

For convenience, let's go back to our **univariate linear regression** model, using just the **temp** for prediction. The following code will run univariate regression multiple times, based upon a **power** term entered into the function. It will then **graph the mse** of the training and testing dataset, compared to the model prediction, as we increase the complexity of the model.


For example...

**If power=2**: The function will run two linear regressions:

* Regression 1: $cnt = m_{temp}x_{temp} + b$
* Regression 2: $cnt = m_{temp}x_{temp} + m_{temp^2}x^{2}_{temp} + b$

In [ ]:
def analyse_linear_reg_fit(data, power=1, target='cnt', random_state=30, chart=True):
    """
    Run linear regression multiple times to analyse the power counts
    
    :param data: <pd.DataFrame>, the data to be fit
    :param power: <list>, the parameters to use as maximum power
    :param target: <str>, the target variable, defaults to 'cnt' assuming the
                   correct preprocessing
    :param random_state: <int>, the random state to use for model selection
    :param chart: <bool>, whether to create a chart for the model
    
    :return lrs: list<LinearRegression>, a list of linear regression objects
    :return train: <pd.DataFrame>, the specific training data
    :return test: <pd.DataFrame>, the specific testing data
    """
    # Create necessary columns
    model_data = data.copy()
    
    # Add columns to fit
    cols = ['temp']
    
    # Go through each column and add the correct powers needed
    if power > 1:
        for p in range(2, power + 1):
            model_data['temp_' + str(p)] = model_data['temp'] ** p
            # append
            cols.append('temp_' + str(p))

    
    # Create a train test split
    train, test = train_test_split(model_data, random_state=random_state)
    
    # Fit data towards each power, and calculate MSE
    powers = []
    mse = []
    mse_type = []
    lrs = []
    
    for i in range(1, power + 1):
        # Run linear regression
        lr = LinearRegression()
        # Fit the model
        lr.fit(train[cols[0:i]], train[target])
        # Get error
        powers += [i, i]
        mse.append(mean_squared_error(train[target], lr.predict(train[cols[0:i]])) / 1e6)
        mse_type.append('Train')
        mse.append(mean_squared_error(test[target], lr.predict(test[cols[0:i]])) / 1e6)
        mse_type.append('Test')
        lrs.append(lr)
    
    # Graph
    if chart:
        mse_df = pd.DataFrame({'Max Power': powers, 'MSE': mse, 'Dataset': mse_type})
        plt.figure(figsize=(10, 5))
        sns.pointplot(x='Max Power', y='MSE', hue='Dataset', data=mse_df, ci=None)
        plt.ylabel('MSE (in millions)')
        plt.title('Train and Test MSE')
    
    return lrs, train, test

Let's use the following cell to run the code, and see what happens.

In [ ]:
# Power
power = 2

# Run the code
_ = analyse_linear_reg_fit(bikeshare_data, power=power, target='cnt')

### Exercise

The **power** variable defined above increases the maximum complexity of the model tested. What should the maximum complexity of this model be based upon the training and testing error?

### Cross validation, overfitting and underfitting

The **maximum power** of the model, that defines whether we are going to use a linear equation of order 1, or 10, is called a **hyperparameter** of the model. A **hyperparemeter** is something that is set _prior_ to model training. Here's a picture to show you the difference between parameters and hyperparameters within an order 2 linear regression model.

---

<img src="img/Parameters_vs_hyperparameters.png" width="600">


---

How do we know the correct hyperparameter to use for our model? We choose hyperparemeters based upon what **generalises** well to data that the model was not specifically trained on. In our previous example, we analysed the **MSE** of a **test dataset** that was **held out** from model training to assess what the complexity of our model should be.

The process we went through has a name, and it's called **cross-validation**. Here's a definition [from AWS](https://docs.aws.amazon.com/machine-learning/latest/dg/cross-validation.html).

> **Cross-validation** is a technique for evaluating ML models by training several ML models on **subsets** of the available input data and evaluating them on the **complementary subset** of the data. Use cross-validation to detect **overfitting**, ie, failing to generalize a pattern.

In our case:

* **subsets** of the available input data were stored in a variable called `train` that held 80% of our full dataset
* The **complementary subset** of the data for evaluating the model was stored in a variable called `test`, which made up 20% of our dataset

What does **overfitting mean**? Let's look at some graphs to find out.

In [ ]:
# Get a sample dataset
samp_data = bikeshare_data.sample(n=20, random_state=42)

# Figures
plt.figure(figsize=(15, 5))

# Subplot
plt.subplot(1, 3, 1)
sns.regplot(x='temp', y='cnt', order=1, data=samp_data, ci=None)
plt.title('Underfit, Order = 1')
plt.xlabel('Normalised Temperature')
plt.ylabel('Count of bikes')

plt.subplot(1, 3, 2)
sns.regplot(x='temp', y='cnt', order=3, data=samp_data, ci=None)
plt.title('OK Fit, Order = 3')
plt.xlabel('Normalised Temperature')
plt.ylabel('')

plt.subplot(1, 3, 3)
sns.regplot(x='temp', y='cnt', order=8, data=samp_data, ci=None)
plt.title('Overfit, Order = 8')
plt.xlabel('Normalised Temperature')
plt.ylabel('')

print('')

### Thought Exercise

I used **order = 3** on the model because that is what I found was best on the test set from the previous exercise. Do you think order = 3 is actually best?

Let's define some more terms.

> An **overfit** model is a model that fits _really well_ on the training dataset used to fit the parameters, but does not generalise well. We saw this when we added **higher orders** to our model. The training MSE decreased, but the test MSE increased. When you read about model overfitting, you might also see that overfit models have **high variance**

> An **underfit** model is a model that _does not fit_ the training data well. This might happen with lower order, or in general, less complicated models. You might also see online that underfit models are said to have **high bias**.

### Side-note, why bias and variance?

Something that often confuses me is why we see the words **bias** and **variance** related to underfitting and overfitting.

The word **variance** comes from statistics, and is often used to describe variation in a process. Where does our algorithm have variation? It has variation in the training/test set we chose. The **model fit really well the training set that was sampled**. If we had chosen a different training set, it would have resulted in a **very different** set of trained parameters from what we chose, and a different error if we had kept the original model fit.

---

<img src='img/High_Variance.png' width='600'>

---

### Thought exercise

Based upon this notion of **high variance**, have we done a good job choosing a test dataset?

#### So why bias?

The reason we use the word **bias** is because underfit algorithms usually are based upon **our own assumptions** about the simplicity of a model. In underfit models, we assume the model has a certain shape, which restricts the model from training well. Thus, we are putting **our own biases** into the model.

There are more mathematical definitions of bias and variance...but we're not going to go there. [Wikipedia does though.](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff)

### A larger issue

Unfortunately, as you decrease the variance of a model, you usually increase the bias in a model, and vice versa. We call this issue the **bias-variance trade-off**.

## Part 2: Figuring out the correct complexity - regularisation

---
[Top](#ML-Week-2---Linear-Regression) | [Previous section](#Part-1:-Introduction-to-cross-validation) | [Next section](#Part-3:-K-Fold-Cross-Validation) | [Bottom](#Thank-you)


So, let's summarise our cross-validation procedure, based upon our toolkit thus far...


```
power = inputted higher order power

Split training/test sets

for each power:
    1. Add columns to data with increase complexity
    2. Fit model
    3. Predict using test set
    4. Save error


Graph test/train errors. Choose order where the test error starts to increase
```

Let's say we try powers up to 8. We would fit **8 different** linear models for cross-validation. Namely...

* Order 1: $cnt = m_{temp}x_{temp} + b$
* ...
* Order 8: $cnt = m_{temp^8}x^8_{temp} + m_{temp^7}x^7_{temp} + ... + m_{temp}x_{temp}  + b$


### Thought Exercise

Let's say we now fit a model with **two variables**, temperature and windspeed. The **first-order** linear model is thus...

$$ cnt = m_{temp}x_{temp} + m_{windspeed}x_{windspeed} + b $$

<table>
    <tr>
        <td><span style="font-size: 14px; text-align: left">Let's say we want to fit a multivariate linear regression model <strong>up to order 8</strong> with these two parameters. How many linear regression models would we need to fit, to check everything?</span></td>
        <td><img src="img/mind_blown.gif" width="500">
    </tr>
</table>

As we increase the number of variables...we exponentially increase the number of potential fits to our model. We haven't even explored **non-linear** spaces yet (for example, we make a new column of temperature * windspeed, or $x_{temp}x_{windspeed}$). Wouldn't it be nice if we didn't have to worry about this arduous process of tuning every potential hyperaparemeter space?

### Regularisation

#### What do model weights represent?

Let's run this multivariate regression, simply with this equation...

$$ cnt = m_{temp}x_{temp} + m_{windspeed}x_{windspeed} + b $$

We'll print out the model coeffcients as well.

In [ ]:
# Run linear regression
lr = LinearRegression()
# Fit the model
lr.fit(bikeshare_data[['temp', 'windspeed']], bikeshare_data['cnt'])
# Get coefficients for the equation
print('Final equation: cnt = %.2f * x_temp + %.2f * x_windspeed + %.2f' % (lr.coef_[0], lr.coef_[1], lr.intercept_))

### Thought exercise

Look at these coefficients and think about the following...

* As the windspeed changes by 1, how much does the count of bikes used change?
* As the temperature changes by 1, how much does the count of bikes used change?
* If both the windspeed and temperature each change by 1, which has more affect on the count of bikes used?

Thus, the **weights affect** how certain variables influence the model. Let's look at the model weights for a **univariate linear regression** of higher order 8.

In [ ]:
# Run the code
lrs, _, _ = analyse_linear_reg_fit(bikeshare_data, power=8, target='cnt', chart=False)

# Prints coefficients
curr = 1
for m in lrs[-1].coef_:
    print('Order, %d: %.2f' %(curr, m))
    curr += 1

Thought...wouldn't it be great if we could implement a way to **shrink the coefficients (weights)** across all parameters just enough, such that we fit the model with the **least amount of complexity necessary** to get the best error?

### Regularisation

We call methods to simplify the model (and find a balance between overfitting/underfitting) **regularisation**. When we **regularise** models, we usually try to find the right balance of complexity to generalise to the best possible model.

For an example, we'll talk today about two types of regularisation for linear regression, specifically **L1 and L2 regularisation**.

#### Re-look at MSE

Remember that in linear regression, we were trying to **find the coefficients $m_i, b$, that minimise the MSE.** Our MSE equation looked something like this for our **univariate linear regression problem**...

$$ MSE = \frac{1}{N}\sum_{j=1}^{N}(y_j - [mx_j + b])^2 $$

This assumed we have...

* N training examples
* 2 parameters ($m$ and $b$) in the model
* The model outputs a prediction based upon the equation $f(x_j) = mx_j + b$

Thus, our problem for linear regression, is find the $m, b$ such that we...

$$ \min \Big (\frac{1}{N}\sum_{j=1}^{N}(y_j - [mx_j + b])^2 \Big ) $$


### L1 Regularisation (lasso regression)


Let's add an additional parameter, called $\alpha$ (prounounced al-fuh) into this minimisation equation. We will add it such that the new equation looks as follows...

$$ \min \Big (\frac{1}{N}\sum_{j=1}^{N}(y_j - [mx_j + b])^2 + \alpha(|m| + |b|) \Big ) $$


#### Side-note...

In case you don't know, $|x|$ is the **absolute value** operator. It makes numbers positive. For example...

* $|-3| = 3$
* $|4| = 4$

### Thought exercise

How do you think $\alpha$ affects $m$ and $b$?

Let's test it out. We'll run two linear regressions, fitting our **8th order model**, with this additional $\alpha$ parameter.


In [ ]:
# Run the code, generalise 8th order moderl
lrs, train, _ = analyse_linear_reg_fit(bikeshare_data, power=8, target='cnt', chart=False)

# Now run the code, using the alpha parameter
from sklearn.linear_model import Lasso
lass_reg = Lasso(alpha=0.1)
lass_reg.fit(X=train[['temp'] + ['temp_' + str(i) for i in range(2, 9)]], y=train['cnt'])

# Make df
coeff_df = pd.DataFrame({'Order': range(1, 9), 'Regular': lrs[-1].coef_, 'Lasso': lass_reg.coef_})
coeff_df[['Order', 'Regular', 'Lasso']]

So...adding alpha **decreases the weights**, and even better, it begins to decrease higher order terms to $~0$. This is because the overall benefit to lowering the MSE is not substantial enough to include the higher order terms. 

We call the linear regression regularisation using an absolute value term **L1 regularisation, or lasso regression**, and you can use the [sklearn Lasso object](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) to run this linear regression.

### L2 Regularisation (ridge regression)

There is one other type of well-known regularisation used in linear regression, called **L2 regularisation, or ridge regression**. The main difference between this regularisation, and what we just did, is that it **squares** the model parameters instead of taking an absolute value, such that our minimisation equation looks like so...

$$ \min \Big (\frac{1}{N}\sum_{j=1}^{N}(y_j - [mx_j + b])^2 + \alpha(m^2 + b^2) \Big ) $$

How do you think ridge regression will affect the model parameters, compared to lasso regression?

Let's take a look! Run the following code, which will again run regular regression, lasso regression, and now add ridge regression. We'll use the [sklearn Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) object to run this.

In [ ]:
# Run the code, generalise 8th order moderl
lrs, train, _ = analyse_linear_reg_fit(bikeshare_data, power=8, target='cnt', chart=False)

# Prints coefficients
lass_reg = Lasso(alpha=0.1)
lass_reg.fit(X=train[['temp'] + ['temp_' + str(i) for i in range(2, 9)]], y=train['cnt'])

# Run ridge regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.1)
ridge_reg.fit(X=train[['temp'] + ['temp_' + str(i) for i in range(2, 9)]], y=train['cnt'])


# Make df
coeff_df = pd.DataFrame({
    'Order': range(1, 9), 
    'Regular': lrs[-1].coef_, 
    'Lasso': lass_reg.coef_,
    'Ridge': ridge_reg.coef_
})
coeff_df[['Order', 'Regular', 'Lasso', 'Ridge']]

### Thought exercise

If we increase the value of alpha, what will happen to our model weights? Will they get bigger or smaller?

## Part 3: K-Fold Cross Validation

---
[Top](#ML-Week-2---Linear-Regression) | [Previous section](#Part-2:-Figuring-out-the-correct-complexity---regularisation) | [Next section](#Part-4:-Try-it-yourself) | [Bottom](#Thank-you)

One more topic prior to heading off for the evening. Let's go back to the thought exercise from before (if you aren't tired of thinking).

> ### Re-thought, thought exercise
> Based upon this notion of high variance, have we done a good job choosing a test dataset?

Let's analyse this. We'll go back to our `analyse_linear_reg_fit` function. There is a parameter in our `analyse_linear_reg_fit` function, called `random_state`, that can be used to **change** how we split-up our test and training set. Let's play with this parameter a bit. How does it affect the consistency of our model performance when we raise the complexity?

In [ ]:
# Power
random_state = 42
# Run the code
_ = analyse_linear_reg_fit(bikeshare_data, power=5, target='cnt', random_state=random_state)

# Power
random_state = 30
# Run the code
_ = analyse_linear_reg_fit(bikeshare_data, power=5, target='cnt', random_state=random_state)

Some craziness occurs...we get **different results** based upon the **specific training and test sets** that we use. There's some stats bucketed-in that can explain this (based upon whether the traiing/testing datasets have the same  distributions across columns), which we're not going to get into. 

What we will do, is try to dig into a method that will help us feel more comfortable in estimating model performance.

### Adding multiple testing sets

To create the training and test sets, we **randomly split the data** into two different datasets. The issue is, the specific split will dictate our model performance, as we saw with the past example.

So how do we know if a model generalises well to any new data? Well...let's think about an analogy. If you conduct a survey, you don't conduct a survey by surveying a single person, you survey a **large group of people**, and hope that the more and more people you survey, the more likely the results reflect the overall population.

![](https://cdn-images-1.medium.com/max/1600/1*WyCRRXiHvPN3k2ZgjXoK_g.png)

We can think of a specific train/test split as a single survey data point. Ideally, we'd love to create **multiple train/test splits**, and look at the results of generating models across these different samples, to be able to get a better idea of how our model will perform on **new data**. Another word for these different splits, are **folds**.

If we train on **k different splits**, we create **k-folds**. We call this overall process **k-fold cross-validation**.

Here's an example of splitting the datasets into **5 different folds**. We choose a fold to hold-out of our model-fitting within each iteration of training. This leads us to create **five different models**. What we can then do is generate a MSE on the specific test dataset held-out, generating a sample of MSE's from each fold. We can then take an average of the MSE's trained across each fold, and describe the final result in terms of the mean and standard deviation of the MSE's.

Here's a visual to help.

<img src="img/K_Fold_Cross_Val.png" width="700">

Note, in this case, we will call **test set** the **validation set**. You might see different definitions of training, test and validation sets online. Typically...

* Training sets fit the model
* Validation sets tune hyperparameters

Let's run some code...the code will run **10-fold** cross-validation, using the [cross_val_score module](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) module. If you wanted to explicitely create K-Folds, you could use the [K Fold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) module. The code will create...

* 10 different splits within the dataset, where each split is 10% of the data
* It will run linear regression on each of these splits, using the MSE as an error metric on the set held out...
* It will then report the mean/std of the MSE, and graph the distributions of the MSE on a plot, per power

In [ ]:
# Import
from sklearn.model_selection import cross_val_score

def lin_regress_w_k_cross_val(data, power=1, target='cnt', chart=True, cv=10):
    """
    Run linear regression with kfold cross validation to analyse error
    
    :param power: <list>, the parameters to use as powers
    :param target: <str>, the target variable, defaults to 'cnt' assuming the
                   correct preprocessing
    :param chart: <bool>, whether to create a chart for the model
    :param cv: <int>, the number of folds
    
    :return lrs: list<LinearRegression>, a list of linear regression objects
    """
    # Create necessary columns
    model_data = data.copy()
    
    # Add columns to fit
    cols = ['temp']
    
    # Go through each column and add the correct powers needed
    if power > 1:
        for p in range(2, power + 1):
            model_data['temp_' + str(p)] = model_data['temp'] ** p
            # append
            cols.append('temp_' + str(p))
    
    # Fit data towards each power, and calculate MSE
    powers = []
    mse = []
    
    # Get linear regression
    lr = LinearRegression()
    
    for i in range(1, power + 1):
        # Run linear regression and cross validation
        mse += (cross_val_score(
            lr, model_data[cols[0:i]], y=model_data[target], scoring='neg_mean_squared_error', cv=cv
        ) * -1).tolist()
        # Get metrics
        powers += [i] * cv
        
    # Graph
    if chart:
        mse_df = pd.DataFrame({'Max Power': powers, 'MSE': mse})
        plt.figure(figsize=(10, 5))
        sns.pointplot(x='Max Power', y='MSE', data=mse_df, ci=68)
        plt.ylabel('MSE')
        plt.title('Cross Validation Score')

In [ ]:
# Run linear regression with k fold cross-validation
lin_regress_w_k_cross_val(bikeshare_data, power=10, target='cnt', chart=True, cv=10)

Once we decide which power we want (in this case, the 2nd power), we would normally train on the **full dataset**. Notice how using the 2nd power makes **qualitative sense with our understanding of temperature** as well!!

## Part 4: Try it yourself

---
[Top](#ML-Week-2---Linear-Regression) | [Previous section](#Part-3:-K-Fold-Cross-Validation) | [Next section](#Thank-you) | [Bottom](#Thank-you)

We've done quite a bit. Let's review...

* We've analysed and seen the concepts of overfitting and underfitting, and come up with a procedure to choose the best model hyperparameters
* We then analysed more automated ways to choose these parameters, using regularisation
* And lastly, we created a broader cross-validation procedure, to try to help us feel comfortable with our results

We'll put it all together with the following function.

In [ ]:
def lin_regress_w_full_cross_val(
    data, 
    cols=['temp'],
    max_powers=[1],
    regression_type=None,
    target='cnt',
    chart=True, 
    cv=10,
    final_regression=0
):
    """
    Run linear regression with kfold cross validation to analyse error. We will be able to choose
    the type of regression, the columns we want within our model, and the maximum power.
    
    :param data: <pd.DataFrame>, the data for our model
    :param cols: list<str>, a list of columns to use in our model
    :param max_powers: list<int>, the max power to use for a corresponding column
    :param regression_type: <str>, either None (regular), 'lasso', or 'ridge'
    :param target: <str>, the target variable, defaults to 'cnt' assuming the
                   correct preprocessing
    :param chart: <bool>, whether to create a chart for the model
    :param cv: <int>, the number of folds
    :param final_regression: <int>, the final regression model to run. Indicates max power
    
    <OPTIONAL RETURN>
    :return lr: LinearRegression, the linear regression final model
    :return model_data: pd.DataFrame, the specific model data used to fit the returned regression model
    """
    # Create necessary columns
    model_data = data.copy()
    
    # Create columns dict
    all_cols = dict()
    all_cols[1] = cols[:]
    
    # Go through each column and add the correct powers needed
    for i in range(len(cols)):
        all_cols
        if max_powers[i] > 1:
            for p in range(2, max_powers[i] + 1):
                if p not in all_cols.keys():
                    all_cols[p] = []
                model_data[cols[i] + '_' + str(p)] = model_data[cols[i]] ** p
                # Append to columns list
                all_cols[p].append(cols[i] + '_' + str(p))
                
    # Add on all keys
    for i in range(2, len(all_cols.keys()) + 1):
        all_cols[i] += all_cols[i - 1]
                    
    # Fit data towards each power, and calculate MSE
    powers = []
    mse = []
    
    # Get linear regression
    if regression_type == 'lasso':
        lr = Lasso(alpha=1.0)
    elif regression_type == 'ridge':
        lr = Ridge(alpha=0.1)
    else:
        lr = LinearRegression()
    
    for i in range(1, max(max_powers) + 1):
        # Run linear regression and cross validation
        mse += (cross_val_score(
            lr, model_data[all_cols[i]], y=model_data[target], scoring='neg_mean_squared_error', cv=cv
        ) * -1).tolist()
        # Get metrics
        powers += [i] * cv
        
    # Graph
    curr = 0
    fig, axs = plt.subplots(len(cols) + 1, 1, figsize=(15, (len(cols) + 1) * 4))
    if chart:
        mse_df = pd.DataFrame({'Max Power': powers, 'MSE': mse})
        sns.pointplot(x='Max Power', y='MSE', data=mse_df, ci=68, ax=axs[curr])
        
    # Final model
    curr += 1
    if final_regression > 0:
        # Fit
        lr.fit(model_data[all_cols[final_regression]], model_data[target])
        
        # Graph
        for i in range(len(cols)):
            sns.regplot(
                x=cols[i], 
                y=target, 
                data=model_data, 
                order=min(final_regression, max_powers[i]), 
                ci=None, 
                ax=axs[curr]
            )
            curr += 1

        return lr, model_data[all_cols[final_regression] + [target]]

### Exercise

Train the following model and run cross-validation. The inputs are as follows:

* `cols` is a list of columns, for example `['temp', 'windspeed', 'hum']`
* `max_powers` is a list of maximum powers to use within the model for each columns, for example `[3, 1, 2]`
* `regression_type` can be `None` for regular regression, `'lasso'`, or `'ridge'`
* `final_regression`, is either 0, or an integer. This dictates the highest power to include on the final regression model, which is returned. 

The final model is returned based upon the power specifically defined by `final_regression`, as well as the final dataset with all columns used.

In [ ]:
# Adjust columns
cols = ['temp', 'windspeed', 'hum']
max_powers = [3, 1, 2]
regression_type = 'ridge'

# Run regression
lr, training_data = lin_regress_w_full_cross_val(
    bikeshare_data, 
    cols=cols,
    max_powers=max_powers,
    regression_type=regression_type,
    target='cnt',
    chart=True, 
    cv=10,
    final_regression=2
)

### Advanced Exercise

Let's introduce another dataset. I scraped the stock prices from Atlassian for a year and saved them to a file. You can easily do this using [Yahoo Finance](https://finance.yahoo.com/quote/TEAM/history?period1=1526911200&period2=1558447200&interval=1d&filter=history&frequency=1d) yourself. The dataset has the following columns:

| Column | Description |
|--------|-------------|
| Date | The date the price was reocrded |
| Close_M1 | The closing stock price one day _prior_ to the current day |
| Volume | The number of shares traded throughout the day |
| Open | The opening price |
| High | The highest price thorughout the day |
| Low | The lowest price throughout the day |
| Close | The closing price that day |

Let's load the data and graph the price for the year.

In [ ]:
# Load data
atlassian_data = pd.read_csv('data/atlassian_data_1_year.csv')

# Graph
plt.figure(figsize=(15, 5))
sns.lineplot(x='Date', y='Close', data=atlassian_data)

Your goal...

Can you use the features to **develop a linear regression model to predict the "Close" price on a given day?** Why might this work well on this dataset but not others? Feel free to create new features.

In [ ]:
# Adjust columns
cols = ['Close_M1']
max_powers = [3]
regression_type = 'ridge'

# Run regression
lr, training_data = lin_regress_w_full_cross_val(
    atlassian_data, 
    cols=cols,
    max_powers=max_powers,
    regression_type=regression_type,
    target='Close',
    chart=True, 
    cv=10,
    final_regression=3
)

## Thank you

[Top](#ML-Week-3---Cross-Validation) | [Previous section](#Part-4:-Try-it-yourself) | [Next section](#Thank-you) | [Bottom](#Thank-you)

That concludes our week 3 lesson. Hopefully you enjoyed :)

### Downloading the notebook

If you would like to retain your work, please follow the following directions:
* On the top of this screen, in the header menu, click "File", then "Download as" and then "Notebook".
* You will need to download [Python 3.7 with Anaconda](https://www.anaconda.com/distribution/#download-section) to use this in the future